<a href="https://colab.research.google.com/github/cidcom/satellite-cloud-removal-dip/blob/main/01-Example-Use-colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [1]:
! git clone https://github.com/cidcom/satellite-cloud-removal-dip

Cloning into 'satellite-cloud-removal-dip'...
remote: Enumerating objects: 57, done.
remote: Counting objects: 100% (57/57), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 57 (delta 12), reused 42 (delta 7), pack-reused 0
Unpacking objects: 100% (57/57), done.


In [2]:
################################################################################
# INSTALL CONDA ON GOOGLE COLAB
################################################################################
! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

--2022-03-10 07:06:56--  https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85055499 (81M) [application/x-sh]
Saving to: ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’

Miniconda3-py37_4.8 100%[===================>]  81.12M   134MB/s    in 0.6s    

2022-03-10 07:06:56 (134 MB/s) - ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’ saved [85055499/85055499]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: - \ done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - asn1crypto==1.3.0=py37_0
    - ca-certificates==2020.1.1=0
    - certifi==2019.11.28=py37_0
    - cffi==1.14.0=py37h2e261b9_0
    - chardet==3.0.4=py37_1003
    - conda-package-handling==1.6.0=py37h

In [4]:
! conda install pytorch-lightning==1.2.0 torch==1.8.1 numpy=1.19.2 rasterio=1.0.21

Solving environment: \ failed with initial frozen solve. Retrying with flexible solve.
Solving environment: \ failed with initial frozen solve. Retrying with flexible solve.

PackagesNotFoundError: The following packages are not available from current channels:

  - pytorch-lightning==1.2.0
  - torch==1.8.1

Current channels:

  - https://repo.anaconda.com/pkgs/main/linux-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/linux-64
  - https://repo.anaconda.com/pkgs/r/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.




In [6]:
!pip install -q torch==1.8.1 torchvision==0.9.0 numpy==1.19.2 rasterio==1.0.21

     |████████████████████████████████| 804.1 MB 2.3 kB/s 
     |████████████████████████████████| 23.2 MB 1.6 MB/s 
     |████████████████████████████████| 14.5 MB 22.0 MB/s 
     |████████████████████████████████| 19.7 MB 15.8 MB/s 
     |████████████████████████████████| 4.3 MB 33.6 MB/s 
     |████████████████████████████████| 60 kB 5.5 MB/s 
     |████████████████████████████████| 82 kB 1.0 MB/s 
     |████████████████████████████████| 98 kB 6.9 MB/s 
ERROR: torchvision 0.11.3 has requirement torch==1.10.2, but you'll have torch 1.8.1 which is incompatible.


In [10]:
!pip install -q pytorch-lightning==1.2.0

     |████████████████████████████████| 813 kB 8.7 MB/s 
     |████████████████████████████████| 134 kB 59.3 MB/s 
     |████████████████████████████████| 829 kB 48.8 MB/s 
     |████████████████████████████████| 5.8 MB 36.3 MB/s 
     |████████████████████████████████| 596 kB 49.8 MB/s 
     |████████████████████████████████| 1.1 MB 40.8 MB/s 
     |████████████████████████████████| 1.1 MB 44.1 MB/s 
     |████████████████████████████████| 4.9 MB 48.7 MB/s 
     |████████████████████████████████| 781 kB 53.3 MB/s 
     |████████████████████████████████| 97 kB 5.3 MB/s 
     |████████████████████████████████| 156 kB 52.6 MB/s 
     |████████████████████████████████| 289 kB 63.6 MB/s 
     |████████████████████████████████| 126 kB 58.0 MB/s 
     |████████████████████████████████| 4.3 MB 60.3 MB/s 
     |████████████████████████████████| 94 kB 2.8 MB/s 
     |████████████████████████████████| 144 kB 50.2 MB/s 
     |████████████████████████████████| 271 kB 55.4 MB/s 
     |█████████████

In [9]:
import sys
sys.path.append('./satellite-cloud-removal-dip/')

from src import *
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['figure.figsize'] = (20, 10)

ModuleNotFoundError: ignored

In [7]:
import torch
torch.__version__

'1.10.0+cu111'

### Example Cloud Image with a Cloud-Shadow Mask

In [ ]:
s2_image = s2_preprocess('data/example_sample/S2/S1_3c120554-3873-4de2-bcc3-23ae00d28c4f_S2_76dac503-47d7-446b-b92d-56f4dd10f8e2_0_0_256x256.tif')

with open('data/example_sample/mask.npy', 'rb') as f:
    mask = np.load(f)

In [ ]:
plt.subplot(1,2,1)
plt.imshow(mask)
plt.subplot(1,2,2)
plt.imshow(s2_image)

### Supporting Data: Sentinel-1 SAR and a yearly Sentinel-2 average from 2019

In [ ]:
s1_image = s1_preprocess('data/example_sample/S1/S1_3c120554-3873-4de2-bcc3-23ae00d28c4f_S2_76dac503-47d7-446b-b92d-56f4dd10f8e2_0_0_256x256.tif',
                         to_db = True)

with open('data/2019_mean.npy', 'rb') as f:
    s2_mean = np.load(f)

In [ ]:
plt.subplot(1,2,1)
plt.imshow(s2_mean)
plt.subplot(1,2,2)
plt.imshow(rgb(s1_image))

### Initialize Model

In [ ]:
my_model = LitDIP()

my_model.set_target([s2_image, s2_mean, s1_image])
my_model.set_mask([mask,
                   np.ones(mask.shape),
                   np.ones(mask.shape)])

In [ ]:
trainer = pl.Trainer(
    max_epochs = 4,            
    checkpoint_callback=False,
    logger=False,
    gpus = [0]
)
trainer.fit(my_model)

In [ ]:
result, _, _ = my_model.output() # the two unused (_) variables correspond to s2_mean reconstruction and s1_image reconstruction

In [ ]:
plt.subplot(1,3,1)
plt.imshow(mask)
plt.axis('off')
plt.title('Cloud Mask')
plt.subplot(1,3,2)
plt.imshow(s2_image)
plt.axis('off')
plt.title('Cloud Mask')
plt.subplot(1,3,3)
plt.imshow(result)
plt.axis('off')
plt.title('Results')